# Preliminary
* Import libraries
* Load data
* Train BlackBox model

In [146]:
import numpy as np
import pandas as pd
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
BASE_PATH = os.path.realpath('..')
DATASETS_DIR = os.path.join(BASE_PATH, 'datasets')
LIB_DIR = os.path.join(BASE_PATH,'lib')
if LIB_DIR not in sys.path:
    sys.path.append(LIB_DIR)

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

In [141]:
import sys

In [3]:
import scipy as sp

In [4]:
from itertools import combinations, islice

In [5]:
from datetime import datetime

In [107]:
!ls {DATASETS_DIR}

bank.csv       bank-names.txt  hepatitis.names
bank-full.csv  hepatitis.data  mango.csv


In [111]:
mango_ds = pd.read_csv(os.path.join(DATASETS_DIR, 'mango.csv'), index_col=0)
mango_ds

,color,firm,smooth,form,fruit
title,,,,,
apple,yellow,False,True,round,True
grapefruit,yellow,False,False,round,True
kiwi,green,False,False,oval,True
plum,blue,False,True,oval,True
toy cube,green,True,True,cubic,False
egg,white,True,True,oval,False
tennis ball,white,False,False,round,False
mango,green,False,True,oval,True


In [152]:
import fca_interp as fcai

In [118]:
mango_bin_ds = pd.DataFrame()
for f in mango_ds.columns:
    if mango_ds[f].dtype==bool:
        mango_bin_ds[f] = mango_ds[f]
    elif mango_ds[f].dtype==np.dtype('O'):
        for v in mango_ds[f].unique():
            mango_bin_ds[f"{f}_{v}"] = mango_ds[f]==v
    else:
        raise TypeError(f'Column DataType {mango_ds[f].dtype} is not supported. Possible variants are [bool, str]')

In [799]:
fcai = reload(fcai)

In [800]:
cntx = fcai.Context(mango_bin_ds)
cntx

Num of objects: 8, Num of attrs: 10
Objects: apple,grapefruit,kiwi,plum,toy cube,...
Attrs: color_yellow,color_green,color_blue,color_white,firm,...
            color_yellow  color_green  color_blue  color_white   firm  smooth  \
apple               True        False       False        False  False    True   
grapefruit          True        False       False        False  False   False   
kiwi               False         True       False        False  False   False   
plum               False        False        True        False  False    True   
toy cube           False         True       False        False   True    True   

            form_round  form_oval  form_cubic  fruit  
apple             True      False       False   True  
grapefruit        True      False       False   True  
kiwi             False       True       False   True  
plum             False       True       False   True  
toy cube         False      False        True  False  

In [801]:
fm = fcai.FormalManager(cntx)

In [802]:
fm.construct_concepts()
fm.get_concepts()




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 90/90 [00:00<00:00, 2008.24it/s]

{Concept 0: (8 objs, 0 attrs),
 Concept 10: (2 objs, 2 attrs),
 Concept 11: (1 objs, 2 attrs),
 Concept 12: (3 objs, 2 attrs),
 Concept 13: (2 objs, 3 attrs),
 Concept 14: (2 objs, 3 attrs),
 Concept 15: (2 objs, 3 attrs),
 Concept 16: (1 objs, 4 attrs),
 Concept 17: (1 objs, 4 attrs),
 Concept 18: (1 objs, 4 attrs),
 Concept 19: (1 objs, 4 attrs),
 Concept 1: (4 objs, 1 attrs),
 Concept 20: (0 objs, 10 attrs),
 Concept 2: (5 objs, 1 attrs),
 Concept 3: (2 objs, 1 attrs),
 Concept 4: (3 objs, 1 attrs),
 Concept 5: (3 objs, 1 attrs),
 Concept 6: (5 objs, 1 attrs),
 Concept 7: (3 objs, 2 attrs),
 Concept 8: (2 objs, 2 attrs),
 Concept 9: (3 objs, 2 attrs)}

In [803]:
fm.construct_lattice()




  0%|          | 0/20 [00:00<?, ?it/s]


100%|██████████| 20/20 [00:00<00:00, 500.47it/s]

In [804]:
cncpts = list(fm.get_concepts())

In [805]:
cncpts[0]

Concept 7: (3 objs, 2 attrs)

In [806]:
print(cncpts[0])
cncpts[0]._low_neighbs[0]

Concept 7: (3 objs, 2 attrs)


Concept 20: (0 objs, 10 attrs)

In [808]:
fm._top_concept

Concept 0: (8 objs, 0 attrs)

In [811]:
fm._top_concept._low_neighbs

(Concept 20: (0 objs, 10 attrs),
 Concept 18: (1 objs, 4 attrs),
 Concept 19: (1 objs, 4 attrs),
 Concept 16: (1 objs, 4 attrs),
 Concept 17: (1 objs, 4 attrs),
 Concept 11: (1 objs, 2 attrs),
 Concept 13: (2 objs, 3 attrs),
 Concept 3: (2 objs, 1 attrs),
 Concept 15: (2 objs, 3 attrs),
 Concept 8: (2 objs, 2 attrs),
 Concept 10: (2 objs, 2 attrs),
 Concept 14: (2 objs, 3 attrs),
 Concept 7: (3 objs, 2 attrs),
 Concept 4: (3 objs, 1 attrs),
 Concept 9: (3 objs, 2 attrs),
 Concept 5: (3 objs, 1 attrs),
 Concept 12: (3 objs, 2 attrs),
 Concept 1: (4 objs, 1 attrs),
 Concept 2: (5 objs, 1 attrs))

In [820]:
fm._bottom_concept._up_neighbs[0]

Concept 7: (3 objs, 2 attrs)

In [819]:
fm._bottom_concept._up_neighbs[0]._low_neighbs

(Concept 20: (0 objs, 10 attrs),
 Concept 18: (1 objs, 4 attrs),
 Concept 16: (1 objs, 4 attrs),
 Concept 13: (2 objs, 3 attrs))